In [1]:
# LOAD PACKAGES
import tensorflow as tf
print(tf.__version__)

import numpy as np
import pandas as pd
from tensorflow import keras

/Users/nityansuman/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


1.8.0


In [2]:
# LOAD DATA
train_file = "data/train.csv"
test_file = "data/test.csv"

train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

In [3]:
train.head(5) # VIEW THE DATA

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
# REMOVE NAME COLUMN SINCE IT IS OF VERY LITTLE USE IN SURVIVAL PREDICTION
# ALSO REMOVE TICKET NUMBER: GENERALLY IT WILL BE A RANDOM NUMBER
# REMOVE CABIN SINCE IT IS RELATED TO CLASS AND EMBARKED INDIRECTLY
train = train.loc[:, [col for col in train.columns if col not in ["Name", "Ticket", "Cabin"]]]

# DONT FORGET TP REMOVE IT FROM TEST DATASET ALSO
test = test.loc[:, [col for col in test.columns if col not in ["Name", "Ticket", "Cabin"]]]

In [6]:
train.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
test.head(5)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [8]:
# LET US ENCODE EMBARKED COLUMN
train["EmbarkedCode"] = train.Embarked.astype("category").cat.codes
# SAME FOR TEST DATA
test["EmbarkedCode"] = test.Embarked.astype("category").cat.codes

# ENCODE SEX COLUMN AS WELL
train["SexCode"] = train.Sex.astype("category").cat.codes
# SAME FOR TEST DATA
test["SexCode"] = test.Sex.astype("category").cat.codes

In [9]:
train.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,EmbarkedCode,SexCode
0,1,0,3,male,22.0,1,0,7.2500,S,2,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,2,0
3,4,1,1,female,35.0,1,0,53.1000,S,2,0
4,5,0,3,male,35.0,0,0,8.0500,S,2,1


In [10]:
test.head(5)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,EmbarkedCode,SexCode
0,892,3,male,34.5,0,0,7.8292,Q,1,1
1,893,3,female,47.0,1,0,7.0000,S,2,0
2,894,2,male,62.0,0,0,9.6875,Q,1,1
3,895,3,male,27.0,0,0,8.6625,S,2,1
4,896,3,female,22.0,1,1,12.2875,S,2,0


In [11]:
# NOW LET US NORMALIZE THE FARE COLUMN
train.Fare = train.Fare / max(train.Fare)
# SAME FOR TEST
test.Fare = test.Fare / max(test.Fare)

In [12]:
train.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,EmbarkedCode,SexCode
0,1,0,3,male,22.0,1,0,0.014151,S,2,1
1,2,1,1,female,38.0,1,0,0.139136,C,0,0
2,3,1,3,female,26.0,0,0,0.015469,S,2,0
3,4,1,1,female,35.0,1,0,0.103644,S,2,0
4,5,0,3,male,35.0,0,0,0.015713,S,2,1


In [13]:
test.head(5)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,EmbarkedCode,SexCode
0,892,3,male,34.5,0,0,0.015282,Q,1,1
1,893,3,female,47.0,1,0,0.013663,S,2,0
2,894,2,male,62.0,0,0,0.018909,Q,1,1
3,895,3,male,27.0,0,0,0.016908,S,2,1
4,896,3,female,22.0,1,1,0.023984,S,2,0


In [14]:
# SELECT COLUMN TO USE AS FEATURES NOW
train = train.loc[:, [col for col in train.columns if col not in ["PassengerId", "Sex", "Embarked"]]]
# DO SAME FOR TEST DATA
test = test.loc[:, [col for col in test.columns if col not in ["PassengerId", "Sex", "Embarked"]]]

In [15]:
train.head(5)

,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedCode,SexCode
0,0,3,22.0,1,0,0.014151,2,1
1,1,1,38.0,1,0,0.139136,0,0
2,1,3,26.0,0,0,0.015469,2,0
3,1,1,35.0,1,0,0.103644,2,0
4,0,3,35.0,0,0,0.015713,2,1


In [16]:
test.head(5)

,Pclass,Age,SibSp,Parch,Fare,EmbarkedCode,SexCode
0,3,34.5,0,0,0.015282,1,1
1,3,47.0,1,0,0.013663,2,0
2,2,62.0,0,0,0.018909,1,1
3,3,27.0,0,0,0.016908,2,1
4,3,22.0,1,1,0.023984,2,0


BUILD MODEL USING MACHINE LEARNING ALGOITHM : DECISION TREE

In [ ]:
# LOAD DECISION TREE CLASSIFIER
